skip residence 
skip aps

In [1]:
#gets data from the college results reddit

from dotenv import load_dotenv
import os
import praw

load_dotenv()

reddit = praw.Reddit(
    client_id = os.getenv("CLIENT_ID"), 
    client_secret = os.getenv("CLIENT_SECRET"), 
    user_agent = os.getenv("USER_AGENT"), 
)

posts = []

for submission in reddit.subreddit("collegeresults").hot(limit=5):
    #print(submission.title)
    #print(submission.selftext)
    posts.append(submission.selftext)


In [2]:
import os
import json
import pandas as pd
from openai import OpenAI

client = OpenAI(
    api_key=os.environ['OPENAI_API_KEY']
    organization ="org-RtJNzeP8PYRZqqkgTM1Uff3r"
)

expected_format = {
    "Gender": "Male/Female",
    "Race": "",
    "Income Bracket": "",
    "Type of School": "",
    "Hooks": "",
    "Intended Major(s)": "",
    "GPA (UW/W)": "",
    "Rank (or percentile)": "",
    "# of Honors/AP/IB/Dual Enrollment/etc.": "",
    "Senior Year Course Load": "",
    "SAT": "1400",
    "ACT": "35",
    "Extra Curriculars": "",
    "Awards": "",
    "Letters of rec": "",
    "Interviews": "",
    "Essays": "",
    "Acceptances": [],
    "Waitlists": [],
    "Rejections": []
}

extracted_data = []
original_posts = []

for post in posts:
    lower_ed = post.lower()
    if(lower_ed.find("accepted") == -1 and lower_ed.find("acceptance") == -1):
        continue
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": """Extract the following details from the post and return in JSON format. Use the following key. Make your best judgement for each.
            Gender: -1(missing data), 0(male), 1(female), 2(other)
            Race: -1(missing data), 0(white), 1(black), 2(hispanic), 3(asian), 4(other)
            Income Bracket: -1(missing data), 0(0-20k), 1(20k-40k), 2(40k-60k), 3(60k-80k), 4(80k-100k), 5(100k-120k), 6(120k-140k), 7(140k-160k), 8(160k-180k), 9(180k-200k), 10(200k+)
            Type of School: -1(missing data), 0(small public), 1(large public), 2(private), 3(international), 4(home school), 5(other)
            Hooks: -1(missing data or none), 0(legacy), 1(athlete), 2(URM), 3(first gen), 4(other)
            Intended Major(s): -1(missing data), 0(STEM), 1(Humanities), 2(Business), 3(Art), 4(Other)
            GPA (UW/W): -1(missing data), 0(0-2.0), 1(2.0-2.5), 2(2.5-3.0), 3(3.0-3.5), 4(3.5-4.0), 5(4.0+)
            Rank (or percentile): -1(missing data), 0(top 5%), 1(top 10%), 2(top 25%), 3(top 50%), 4(top 100%)
            # of Honors/AP/IB/Dual Enrollment/etc.: -1(missing data), 0(0-5), 1(5-10), 2(10-15), 3(15-20), 4(20+)
            Senior Year Course Load: -1(missing data), 0(Difficulty 1-3), 1(Difficulty 4-6), 2(Difficulty 7-9), 3(Difficulty 10)
            SAT: -1(missing data), 0(1000-1100), 1(1100-1200), 2(1200-1300), 3(1300-1400), 4(1400-1500), 5(1500-1600)
            ACT: -1(missing data), 0(10-20), 1(21-25), 2(26-30), 3(31-33), 4(34-35), 5(36)
            Extra Curriculars: [SCORE HERE FROM 1-10 ON HOW IMPRESSIVE, DIFFICULT, OR TIME-CONSUMING THE EC'S ARE. -1 if missing data]
            Awards: [SCORE HERE FROM 1-10 ON HOW IMPRESSIVE, DIFFICULT, OR TIME-CONSUMING THE AWARDS ARE. -1 if missing data]
            Letters of rec: [SCORE HERE FROM 1-10 ON HOW STRONG THE LETTERS OF REC ARE. -1 if missing data]
            Interviews: [SCORE HERE FROM 1-10 ON HOW STRONG THE INTERVIEWS ARE. -1 if missing data]
            Essays: [SCORE HERE FROM 1-10 ON HOW STRONG THE ESSAYS ARE. -1 if missing data]
            Acceptances: [LIST OF COLLEGES ACCEPTED TO]
            Waitlists: [LIST OF COLLEGES WAITLISTED TO]
            Rejections: [LIST OF COLLEGES REJECTED FROM]
            """},
            {"role": "user", "content": f"""
            Post:
            {post}
            Extract the following details and return in JSON format:
            {json.dumps(expected_format, indent=2)}
            """}
        ],
        temperature=0,
        response_format={"type": "json_object"}
    )
    
    extracted_data.append(response.choices[0].message.content)
    original_posts.append(post)

extracted_data_dicts = [json.loads(data) for data in extracted_data]
df = pd.DataFrame(extracted_data_dicts)

df['Original Post'] = original_posts

df.to_csv('data_new_clean_test.csv', index=False)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}